# Microsoft Azure AutoML Demo

Azure ML & Azure Databricks notebooks by Parashar Shah.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Purpose and Challenge

The purpose of this notebook is for the user to build and deploy a Machine Learning (ML) application using Azure Machine Learning (AML) service. It is a predictive maintenance scenario based on https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan.

This notebook has the complete code to load, prep, train and deploy the model. We chose a small public data set for this demo so as to run the entire process in only few minutes.

Following are the high level steps:

1. Create AML Workspace
2. Acquire and Prepare Data
3. Automated ML
4. Deploy Model as webservice
5. Predictions

## 1. Create cluster (in this lab it is pre-created)

Please follow the instructions from Microsoft documentation with your customers https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment#azure-databricks

## 2. Acquire and Prepare Data
For this notebook, we will use the NASA Prognostics Center's Turbo-Fan Failure dataset.  It is located here: https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan

Download and un-zip the data

In [8]:
import logging
import os
import random
import time

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

In [9]:
# import needed libraries for downloading and unzipping the file
import urllib.request
from zipfile import ZipFile

In [10]:
# download from url
response = urllib.request.urlopen("https://ti.arc.nasa.gov/c/6/")
output = open('CMAPSSData.zip', 'wb')    # note the flag:  "wb"        
output.write(response.read())
output.close()

In [11]:
# unzip files
zipfile = ZipFile("CMAPSSData.zip")
zipfile.extract("train_FD001.txt")

Out[ 4 ]: '/databricks/driver/train_FD001.txt'

Next we read our data into a Pandas DataFrame.
Note the headers were not in the space seperated txt file, so we assign them from the ReadMe in the zip file. In pandas we use read_csv with the delimiter option.

In [13]:
train = pd.read_csv("train_FD001.txt", delimiter="\s|\s\s", index_col=False, engine='python', names=['unit','cycle','os1','os2','os3','sm1','sm2','sm3','sm4','sm5','sm6','sm7','sm8','sm9','sm10','sm11','sm12','sm13','sm14','sm15','sm16','sm17','sm18','sm19','sm20','sm21'])

Take a quick look at the data

In [15]:
train.head(10)

Out[ 6 ]: 
 unit cycle os1 os2 ... sm18 sm19 sm20 sm21
0 1 1 -0.0007 -0.0004 ... 2388 100.0 39.06 23.4190
1 1 2 0.0019 -0.0003 ... 2388 100.0 39.00 23.4236
2 1 3 -0.0043 0.0003 ... 2388 100.0 38.95 23.3442
3 1 4 0.0007 0.0000 ... 2388 100.0 38.88 23.3739
4 1 5 -0.0019 -0.0002 ... 2388 100.0 38.90 23.4044
5 1 6 -0.0043 -0.0001 ... 2388 100.0 38.98 23.3669
6 1 7 0.0010 0.0001 ... 2388 100.0 39.10 23.3774
7 1 8 -0.0034 0.0003 ... 2388 100.0 38.97 23.3106
8 1 9 0.0008 0.0001 ... 2388 100.0 39.05 23.4066
9 1 10 -0.0033 0.0001 ... 2388 100.0 38.95 23.4694

[10 rows x 26 columns]

Our dataset has a number of units in it, with each engine flight listed as a cycle. The cycles count up until the engine fails. What we would like to predict is the no. of cycles until failure. 
So we need to calculate a new column called RUL, or Remaining Useful Life.  It will be the last cycle value minus each cycle value per unit.

In [17]:
# Assign ground truth
def assignrul(df):
    maxi = df['cycle'].max()
    df['rul'] = maxi - df['cycle']
    return df
    

train_new = train.groupby('unit').apply(assignrul)

train_new.columns

Out[ 7 ]: 
Index(['unit', 'cycle', 'os1', 'os2', 'os3', 'sm1', 'sm2', 'sm3', 'sm4', 'sm5',
 'sm6', 'sm7', 'sm8', 'sm9', 'sm10', 'sm11', 'sm12', 'sm13', 'sm14',
 'sm15', 'sm16', 'sm17', 'sm18', 'sm19', 'sm20', 'sm21', 'rul'],
 dtype='object')

Now our dataframe has the 'RUL' column.  Predicting this value will be the objective of this exercise.

In [19]:
train_new.head(5)

Out[ 8 ]: 
 unit cycle os1 os2 os3 ... sm18 sm19 sm20 sm21 rul
0 1 1 -0.0007 -0.0004 100.0 ... 2388 100.0 39.06 23.4190 191
1 1 2 0.0019 -0.0003 100.0 ... 2388 100.0 39.00 23.4236 190
2 1 3 -0.0043 0.0003 100.0 ... 2388 100.0 38.95 23.3442 189
3 1 4 0.0007 0.0000 100.0 ... 2388 100.0 38.88 23.3739 188
4 1 5 -0.0019 -0.0002 100.0 ... 2388 100.0 38.90 23.4044 187

[5 rows x 27 columns]

First note that the sensor measurements do seem to be changing as we near 0 RUL. This implies that we should be able to make a model that will be useful enough for business value.

We are now ready to train a model on this data using Automated ML.

## 3. Azure Automated ML

Here we utilize Azure's AutoML package to automate the scaling of the sensors, selection of sensors, and automatically train and evaluate many different types of ML models.

In [23]:
import azureml.core

# Check core SDK version number - based on build number of preview/master.
print("SDK version:", azureml.core.VERSION)

username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user').split("@")[0]
print("Your username is {0}".format(username))

SDK version: 1.0.18
Your username is seretkow

In [24]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

Provide your Machine Learning Workspace credentials to run AutoML. You will need to perform Microsoft's MFA. Please follow the manual auth instructions.

In [26]:
subscription_id = "70b8f39e-8863-49f7-b6ba-34a80799550c" #you should be owner or contributor
resource_group = "mlserviceresourcegroup" #you should be owner or contributor
workspace_name = "MLServiceWorkspace" #your workspace name
workspace_region = "West Europe" #your region

You can have more options when creating Workspace

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py has more options.

In [28]:
# Import the Workspace class and check the Azure ML SDK version.
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)
ws.get_details()

Out[ 12 ]: 
{'containerRegistry': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/mlserviceresourcegroup/providers/microsoft.containerregistry/registries/mlserviceworks2334900829',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/mlserviceresourcegroup/providers/microsoft.storage/storageaccounts/mlserviceworks0764555666',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityPrincipalId': 'b4335a6a-b27f-4b38-9f49-52ed442de49e',
 'description': '',
 'id': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/mlserviceresourcegroup/providers/Microsoft.MachineLearningServices/workspaces/MLServiceWorkspace',
 'friendlyName': '',
 'keyVault': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/mlserviceresourcegroup/providers/microsoft.keyvault/vaults/mlserviceworks0657942370',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'name': 'MLServiceWorkspace',
 'creationTime': '2018-11-14T09:30:26.3955776+00:00',
 'applicationInsights': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/mlserviceresourcegroup/providers/microsoft.insights/components/mlserviceworks9069414308',
 'workspaceid': 'f5bb1783-07f8-419d-a006-faaadbf5a5db'}

In [29]:
from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config(path="/databricks/driver/aml_config/", file_name="config_{0}.json".format(username))

Wrote the config file config_seretkow.json to: /databricks/driver/aml_config/config_seretkow.json

In [30]:
# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-predictive-rul'
project_folder = './sample_projects/automl-demo-predmain'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

Out[ 14 ]: 
 
Experiment Name automl-predictive-rul 
Location westeurope 
Project Directory ./sample_projects/automl-demo-predmain
Resource Group mlserviceresourcegroup 
SDK version 1.0.18 
Subscription ID 70b8f39e-8863-49f7-b6ba-34a80799550c 
Workspace Name MLServiceWorkspace

In [31]:
# put training data into X and Y df
X_train = train_new.drop(['rul'], axis=1)[2:]
y_train = train_new[['rul']][2:]

In [32]:
X_train.head(5)

Out[ 16 ]: 
 unit cycle os1 os2 os3 ... sm17 sm18 sm19 sm20 sm21
2 1 3 -0.00 0.00 100.00 ... 390 2388 100.00 38.95 23.34
3 1 4 0.00 0.00 100.00 ... 392 2388 100.00 38.88 23.37
4 1 5 -0.00 -0.00 100.00 ... 393 2388 100.00 38.90 23.40
5 1 6 -0.00 -0.00 100.00 ... 391 2388 100.00 38.98 23.37
6 1 7 0.00 0.00 100.00 ... 392 2388 100.00 39.10 23.38

[5 rows x 26 columns]

In [33]:
y_train.head(5)

Out[ 17 ]: 
 rul
2 189
3 188
4 187
5 186
6 185

In [34]:
X_test = train_new.drop(['rul'], axis=1)[0:1]
y_test = train_new[['rul']][0:1]
print (X_test)
print (y_test)

unit cycle os1 os2 os3 ... sm17 sm18 sm19 sm20 sm21
0 1 1 -0.00 -0.00 100.00 ... 392 2388 100.00 39.06 23.42

[1 rows x 26 columns]
 rul
0 191

In [35]:
import azureml.dataprep as dprep
import uuid

X_dflow = dprep.read_pandas_dataframe(X_train, temp_folder='/dbfs/tmp'+str(uuid.uuid4()))
y_dflow = dprep.read_pandas_dataframe(y_train, temp_folder='/dbfs/tmp'+str(uuid.uuid4()))

In [36]:
y_dflow.get_profile()

Out[ 20 ]: 
ColumnProfile:
 column_name: rul
 type: FieldType.INTEGER

 min: 0.0
 max: 361.0
 count: 20629.0
 missing_count: 0.0
 not_missing_count: 20629.0
 percent_missing: 0.0
 error_count: 0.0
 empty_count: 0.0


 Quantiles:
 0.1%: 0.0
 1%: 20.12454935193565
 5%: 19.23936170212766
 25%: 50.991157863451086
 50%: 102.58762669969065
 75%: 155.34728036069126
 95%: 228.81278684550742
 99%: 287.4373469387753
 99.9%: 341.9720666666661

 mean: 107.79984487856936
 std: 68.8795162043496
 variance: 4744.387752545258
 skewness: 0.5000932148957241
 kurtosis: -0.21850756566908247

Now we are ready to configure Azure Automated ML.  We provide necessary information on: what we want to predict, what accuracy metric we want to use, how many models we want to try, and many other parameters. Automated ML will also automatically scale the data for us.

## Configure Automated ML

You can use these params.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**spark_context**|Spark Context object. for Databricks, use spark_context=sc|
|**max_concurrent_iterations**|Maximum number of iterations to execute in parallel. This should be <= number of worker nodes in your Azure Databricks cluster.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification. This should be an array of integers.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|
|**preprocess**|set this to True to enable pre-processing of data eg. string to numeric using one-hot encoding|
|**exit_score**|Target score for experiment. It is associated with the metric. eg. exit_score=0.995 will exit experiment after that|

In [39]:
automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors_regression.log',
                             primary_metric = 'r2_score',
                             iteration_timeout_minutes = 5, #some runs may take 10+ mins hence limiting it for workshop
                             iterations = 10, #you may change this to a higher number and see what happens
                             #validation_size = 0.20, #for large datasets only and not needed for workshop
                             verbosity = logging.INFO,
                             max_concurrent_iterations = 2, #change it based on number of worker nodes
                             spark_context=sc, #databricks/spark related
                             n_cross_validations = 3, #(only needed for small datasets and if validation size is not set)
                             X = X_dflow,
                             y = y_dflow,
                             preprocess=True, #preprocess
                             path = project_folder)

Finally we are ready to submit the experiment to Automated ML service. This step can take longer depending on the settings. AutoML will give us updates as models are trained and evaluated by the metric we specified above. The information from each ML model training will be stored in the Experiment section of the Azure ML Workspace in Azure Portal.

In [41]:
local_run = experiment.submit(automl_config, show_output = True) # for higher runs please use show_output=False and use the below

In [42]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Portal: {}</a>".format(local_run.get_portal_url(), local_run.id))

Your experiment in Azure Portal: AutoML_216baaf6-b766-4771-bbc4-e10ed0c96d74

In [43]:
# run this only after the portal shows experiment 'Completed'
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}    
    metricslist[int(properties['iteration'])] = metrics
    hyperparamproperties = run.get_properties()
    print(hyperparamproperties['run_properties'])

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

Out[ 24 ]: 
Empty DataFrame
Columns: []
Index: []

We want to keep the best model and deploy it as a service.

In [45]:
# find the run with the highest accuracy value.
best_run, fitted_model = local_run.get_output()
print(best_run)

--------------------------------------------------------------------------- 
 Exception Traceback (most recent call last)
 <command-691515439878460> in <module> () 
 1 # find the run with the highest accuracy value. 
 ----> 2 best_run , fitted_model = local_run . get_output ( ) 
 3 print ( best_run ) 

 /databricks/python/lib/python3.5/site-packages/azureml/train/automl/run.py in get_output (self, iteration, metric) 
 395 
 396 if curr_run is None : 
 --> 397 raise Exception ( "Could not find model with valid score for metric '{0}'" . format ( metric ) ) 
 398 
 399 curr_run . download_file ( name = model_name , output_file_path = model_local ) 

 Exception : Could not find model with valid score for metric 'r2_score'

## 4. Deploy Model

In [47]:
# register model in workspace & use the same in your score.py file
description = 'AutoML-RUL-Regression-20190219'
tags = None
model=local_run.register_model(description=description, tags=tags)
local_run.model_id # Use this id to deploy the model as a web service in Azure. Update score file with the output.

After we register the model in our AML Workspace, it should be visible in Azure Portal.

Now we want to deploy the model as a REST API (real time webservice) that we can feed a row or rows of "X" data to, and return the predicted 'RUL' value.  To accomplish this, we will build a container image in our AML Workspace and deploy that image as a Container instance in Azure's ACI service.  We will then obtain an IP address where we can submit data and receive back the predicted 'RUL' value.

There are 3 things we need: 
1. A score.py file that contains the init() and run() functions with instructions on how to load and score with the model. Update the model name in this file.
2. A mydeployenv.yml file that contains information on the python environment in which the model needs to run
3. Configurations for our images and our services, using functions provided by AzureML service.

The cells below help you set these up.

In [49]:
scorefilename = (('score'+str(uuid.uuid4()))[0:10]) + ".py"
print(scorefilename) #change the filename in score file

In [50]:
%%writefile score270c9.py
# Change the name based on the randomly generated filename
# Scoring Script will need model id from registered model
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

import azureml.train.automl

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('AutoML820abfb1abest') # update this based on previously registered model
    print(model_path)
    model = joblib.load(model_path)
    

def run(raw_data):
    # grab and prepare the data
    data = (np.array(json.loads(raw_data)['data'])).reshape(1,-1)
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

In [51]:
condafilename = (('mydeploy'+str(uuid.uuid4()))[0:14]) + ".yml"
print(condafilename) #change the filename in score file

In [52]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'], pip_packages=['azureml-sdk[automl]'])

conda_env_file_name = condafilename
myenv.save_to_file('.', conda_env_file_name)

In [53]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=5, 
                                               tags={"data": "RUL",  "method" : "sklearn"}, 
                                               description='Predict RUL with Azure AutoML')

Finally, configure the container image and deploy the service. Make sure the filenames match, your Workspace is in variable ws, and your model name is correct. It will create your containter image and deploy it as a webservice.

This process can take up to 10 minutes, so please be patient. You can check the progress bar periodically ...

In [55]:
# this will take 10-15 minutes to finish

service_name = "rul-pred" #change this to whatever other name you want
runtime = "python" 
driver_file = scorefilename #use the name generated earlier
my_conda_file = conda_env_file_name #use the name generated earlier

# image creation
from azureml.core.image import ContainerImage
myimage_config = ContainerImage.image_configuration(execution_script = driver_file, 
                                    runtime = runtime, 
                                    conda_file = my_conda_file)

# Webservice creation
myservice = AciWebservice.deploy_from_model(
  workspace=ws, 
  name=service_name,
  deployment_config = aciconfig,
  models = [model],
  image_config = myimage_config
    )

myservice.wait_for_deployment(show_output=True)

Just as a check, we can retrieve the URL for the scoring function.

In [57]:
print(myservice.scoring_uri)

Let's check to see if the service is working.  Here we submit a single row of data from X_train to see if it returns a reasonable prediction.

In [59]:
test = X_test.values.tolist()
testlabel = y_test.values

In [60]:
import requests
import json

# send a random row from the test set to score
#random_index = np.random.randint(0, len(X_train)-1)
input_data = "{\"data\": " + str(test) + "}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(myservice.scoring_uri, input_data, headers=headers)

print("POST to url", myservice.scoring_uri)
print("input data:", input_data)
print("label:", testlabel)
print("prediction:", resp.text)

Here we see one engine evolving through many flights, or cycles.  As we approach failure, the rul declines to zero, as does the prediction.  This is a good example of how the predictive model can assist in estimate the future failure of the engine.

Note that the model does not perform well at high rul.  This is an acceptable outcome as the engine is far from failure.

To avoid any run-away Azure costs, we always delete un-necessary services when we are done.

In [63]:
myservice.delete()

You may deploy this docker image to AKS.

## 5. Conclusions

We have executed an end-to-end Azure ML Service project with a real life example. We started with a problem at hand, created an Azure ML Workspace, downloaded a predictive maintenance dataset, processed the data, train a sophisticated model with Azure Automated ML, and deployed that model quickly and easily to ACI (AKS) using Azure's Machine Learning service.